In [4]:
pip install tavily-python

  Using cached tavily_python-0.7.11-py3-none-any.whl.metadata (7.5 kB)
Using cached tavily_python-0.7.11-py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
##import required libraries
import os
from typing import Annotated
from tavily import TavilyClient
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json, register_function
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
config_list=[
    {"model":"Qwen3",
     "base_url":"http://localhost:11434/v1",
     "api_key":"ollama",
     "price":[0,0]},
]
llm_config={'config_list':config_list}

In [3]:
tavily=TavilyClient(api_key=os.getenv('TAVILY_API_KEY'))

def search_tool(query: Annotated[str, "The search query"])->Annotated[str, "The search result"]:
    return tavily.get_search_context(query=query,search_depth="advanced")

Adding ReAct prompt

In [4]:
ReAct_prompt="""
Answer the following question as best you can. You have access to tool provided.

Use the following format:

Question: the input question you must answer
Thought:You should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result to the action
... (this process can repeat multiple times)
Thought: I know the final answer
Final Answer: the final answer to the original input question


Begin!
Question:{input}
"""

#Define the ReAct prompt message. Assuming a "question" field is present in the context
def react_prompt_message(sender, recipient, context):
    return ReAct_prompt.format(input=context["question"])

Define the Agent

In [5]:
user_proxy=UserProxyAgent(
    name="User",
    is_termination_msg=lambda x:x.get("content","") and x.get("content","").rstrip().endswith("TERMINATE"),
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=10,
    code_execution_config=False
)

research_assistant=AssistantAgent(
    name="Assistant",
    system_message="""
    You are a helpful research assistant who has the ability to search the web using the provided tools. Only use the tools you have been provided with. Reply TERMINATE when the task is done""",
    llm_config=llm_config,
)

Register the Search Tool

In [6]:
#Register the search tool.
register_function(
    search_tool,
    caller=research_assistant,
    executor=user_proxy,
    name="Search_Tool",
    description="Search the web for the given query",
)

Initiate the chat

In [9]:
user_proxy.initiate_chats(
    [
        { "recipient":research_assistant,
        "messages":react_prompt_message,
        "question":"Who won the T20 cricket world cup in 2024"
}
]
   )


********************************************************************************
Starting a new chat....

********************************************************************************
User (to Assistant):

Who won the cricket world cup in 2019

--------------------------------------------------------------------------------
Assistant (to User):

<think>
Okay, the user is asking who won the 2019 cricket world cup. I need to find the correct answer. Let me think about the tools I have. There's the Search_Tool that can search the web. I should use that. The function requires a query parameter. The query here would be "Who won the cricket world cup in 2019". I'll call the Search_Tool with that query. Once I get the results, I can extract the information. From what I remember, England won in 2019, but I should confirm with the search tool to make sure. Alright, let's make the tool call.
</think>


***** Suggested tool call (call_ym96qlgf): Search_Tool *****
Arguments: 
{"query":"Who won

[ChatResult(chat_id=None, chat_history=[{'content': 'Who won the cricket world cup in 2019', 'role': 'assistant', 'name': 'User'}, {'content': '<think>\nOkay, the user is asking who won the 2019 cricket world cup. I need to find the correct answer. Let me think about the tools I have. There\'s the Search_Tool that can search the web. I should use that. The function requires a query parameter. The query here would be "Who won the cricket world cup in 2019". I\'ll call the Search_Tool with that query. Once I get the results, I can extract the information. From what I remember, England won in 2019, but I should confirm with the search tool to make sure. Alright, let\'s make the tool call.\n</think>\n\n', 'tool_calls': [{'id': 'call_ym96qlgf', 'function': {'arguments': '{"query":"Who won the cricket world cup in 2019"}', 'name': 'Search_Tool'}, 'type': 'function', 'index': 0}], 'role': 'assistant'}, {'content': 'yes England won 2019 world cup', 'tool_responses': [{'role': 'tool', 'tool_cal